In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "codellama/CodeLlama-13b-Instruct-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...


Loading checkpoint shards:  33%|████████████████████▎                                        | 1/3 [00:22<00:44, 22.12s/it]


Loading checkpoint shards:  67%|████████████████████████████████████████▋                    | 2/3 [00:50<00:26, 26.01s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [01:02<00:00, 19.38s/it]


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████| 3/3 [01:02<00:00, 20.78s/it]

codellama/CodeLlama-13b-Instruct-hf does not have a padding token! Will use pad_token = <unk>.


In [5]:
lora_rank = 16

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.3 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

In [ ]:
import random
from datasets import DatasetDict, concatenate_datasets

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world""",
]

PROMPTS = [
    """The given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.
    
    This is the vulnerable functions:
    ```solidity
    {code}
    ```
    
This is the vulnerability explanation: 
{explanation}

Recommendation:
""",
]


def make_conversational(examples, system_prompt, prompt):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    recom = examples['vuln_recommendation']
    selected_system_prompt = system_prompt
    selected_user_prompt = prompt
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code, explanation=desc)},
                    {"role": "assistant", "content": recom}]
    
    return { "conversations" : conversation}

# List of splits you want to process
splits = ['train', 'val', 'test']

# Dictionary to store concatenated datasets for each split
concatenated_datasets = {split: [] for split in splits}

system_prompt = SYSTEM_PROMPT[0]
prompt = PROMPTS[0]

# Create a new dataset using map
new_dataset = dataset.map(lambda ex: make_conversational(ex, system_prompt, prompt))

# Append the correct split of the new dataset to the corresponding list
for split in splits:
    concatenated_datasets[split].append(new_dataset[split])

# Now concatenate all datasets for each split
dataset = DatasetDict({
    split: concatenate_datasets(concatenated_datasets[split]).shuffle(seed=42) for split in splits
})

dataset = dataset.remove_columns(["vuln_title", "severity"])

In [8]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset['train'][5]["conversations"]

[{'content': 'You are the smartest AI solidity smart contract security auditor in the world',
  'role': 'system'},
 {'content': 'The given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.\n    \n    This is the vulnerable functions:\n    ```solidity\n    function less(ERC4626 vault, uint256 feiAmount) external nonReentrant requiresLocalOrMasterAuth {\n        // Update the total Fei deposited into the Vault proportionately.\n        getTotalFeiBoostedForVault[vault] -= feiAmount;\n\n        unchecked {\n            // Decrease the boost total proportionately.\n            // Cannot underflow because the total cannot be less than a single Vault.\n            totalFeiBoosted -= feiAmount;\n        }\n\n        emit VaultLessened(msg.sender, vault, feiAmount);\n\n        // Withdraw the specified amount of Fei from the Vault.\n        vault.withdraw(feiAmount, address(this), address(this));\n\

In [10]:
dataset['train'][5]["text"]

'<s>[INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world\n<</SYS>>\n\nThe given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.\n    \n    This is the vulnerable functions:\n    ```solidity\n    function less(ERC4626 vault, uint256 feiAmount) external nonReentrant requiresLocalOrMasterAuth {\n        // Update the total Fei deposited into the Vault proportionately.\n        getTotalFeiBoostedForVault[vault] -= feiAmount;\n\n        unchecked {\n            // Decrease the boost total proportionately.\n            // Cannot underflow because the total cannot be less than a single Vault.\n            totalFeiBoosted -= feiAmount;\n        }\n\n        emit VaultLessened(msg.sender, vault, feiAmount);\n\n        // Withdraw the specified amount of Fei from the Vault.\n        vault.withdraw(feiAmount, address(this), address(this));\n\n        // Get ou

In [11]:
tokenized = dataset['train'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
# print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Train Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['train'] = dataset['train'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Train Max Length =  1548


In [12]:
tokenized = dataset['val'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Val Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['val'] = dataset['val'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Val Max Length =  1676


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

train_dataset = dataset['train']
val_dataset = dataset['val']

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, 
    args = TrainingArguments(
        eval_strategy="steps",
        eval_steps=500,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, 
        warmup_ratio = 0.1,
        num_train_epochs = 5, 
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", 
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        load_best_model_at_end=True,
        output_dir = "outputs_vuln_exp_codellama",
        report_to = "none", 
    ),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<</SYS>>\n\n",
    response_part = "[/INST]",
)

In [15]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<s><s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world\n<</SYS>>\n\nThe given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.\n    \n    This is the vulnerable functions:\n    ```solidity\n    constructor(address gatewayImplementation, bytes memory params) {\n        _setAddress(KEY_IMPLEMENTATION, gatewayImplementation);\n\n        (bool success, ) = gatewayImplementation.delegatecall(\n            abi.encodeWithSelector(IAxelarGateway.setup.selector, params)\n        );\n\n        if (!success) revert SetupFailed();\n    }\n    ```\n    \nThis is the vulnerability explanation: \nAs written in the solidity documentation, the low-level functions call, delegatecall and staticcall return true as their first return value if the account called is non-existent, as part of the design of the EVM. Account existence must be checked prior to calling if 

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [17]:
tokenizer.decode(trainer.eval_dataset[5]["input_ids"])

'<s><s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world\n<</SYS>>\n\nThe given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.\n    \n    This is the vulnerable functions:\n    ```solidity\n    function deposit(uint _amount) external {\n        settlePendingObligation();\n        // we want to protect new LPs, when the insurance fund is in deficit\n        require(pendingObligation == 0, "IF.deposit.pending_obligations");\n\n        uint _pool = balance();\n        uint _totalSupply = totalSupply();\n        if (_totalSupply == 0 && _pool > 0) { // trading fee accumulated while there were no IF LPs\n            vusd.safeTransfer(governance, _pool);\n            _pool = 0;\n        }\n\n        vusd.safeTransferFrom(msg.sender, address(this), _amount);\n        uint shares = 0;\n        if (_pool == 0) {\n            shares = _amount;\n        }

In [18]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.eval_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 595 | Num Epochs = 5 | Total steps = 2,975
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 62,586,880/13,000,000,000 (0.48% trained)


Step,Training Loss,Validation Loss
500,1.796900,1.413581
1000,1.250800,1.398486
1500,1.761000,1.411695
2000,0.436500,1.457904
2500,0.145700,1.506624


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
model.save_pretrained("codellama13b_recommendation")  
tokenizer.save_pretrained("codellama13b_recommendation")

('codellama13b_recommendation/tokenizer_config.json',
 'codellama13b_recommendation/special_tokens_map.json',
 'codellama13b_recommendation/tokenizer.model',
 'codellama13b_recommendation/added_tokens.json',
 'codellama13b_recommendation/tokenizer.json')

In [21]:
test_dataset = dataset['test']
test_dataset = test_dataset.map(lambda row: {'answer':row['conversations'][-1]})
test_dataset = test_dataset.map(lambda row: {'conversations':row['conversations'][:-1]})

In [22]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = test_dataset['conversations'][1]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 8192, use_cache = True,
                         temperature = 0.1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<s> [INST] <<SYS>>\nYou are the smartest AI solidity smart contract security auditor in the world\n<</SYS>>\n\nThe given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.\n    \n    This is the vulnerable functions:\n    ```solidity\n    function withdrawWithTimelockFrom(\n    address from,\n    uint256 amount,\n    address controlledToken\n  )\n    external override\n    nonReentrant\n    onlyControlledToken(controlledToken)\n    returns (uint256)\n  {\n    uint256 blockTime = _currentTime();\n    (uint256 lockDuration, uint256 burnedCredit) = _calculateTimelockDuration(from, controlledToken, amount);\n    uint256 unlockTimestamp = blockTime.add(lockDuration);\n    _burnCredit(from, controlledToken, burnedCredit);\n    ControlledToken(controlledToken).controllerBurnFrom(_msgSender(), from, amount);\n    _mintTimelock(from, amount, unlockTimestamp);\n    emit TimelockedWithdrawal(_msgSende

In [23]:
decoded_outputs = tokenizer.batch_decode(outputs)

In [24]:
test_dataset['answer'][1]

{'content': 'Recommend that the unlock timestamp should be increased by duration each time, instead of being reset to the duration.',
 'role': 'assistant'}

In [25]:
test_dataset['vuln_code'][1]

'function withdrawWithTimelockFrom(\n    address from,\n    uint256 amount,\n    address controlledToken\n  )\n    external override\n    nonReentrant\n    onlyControlledToken(controlledToken)\n    returns (uint256)\n  {\n    uint256 blockTime = _currentTime();\n    (uint256 lockDuration, uint256 burnedCredit) = _calculateTimelockDuration(from, controlledToken, amount);\n    uint256 unlockTimestamp = blockTime.add(lockDuration);\n    _burnCredit(from, controlledToken, burnedCredit);\n    ControlledToken(controlledToken).controllerBurnFrom(_msgSender(), from, amount);\n    _mintTimelock(from, amount, unlockTimestamp);\n    emit TimelockedWithdrawal(_msgSender(), from, controlledToken, amount, unlockTimestamp);\n\n    // return the block at which the funds will be available\n    return unlockTimestamp;\n  }\n\nfunction _mintTimelock(address user, uint256 amount, uint256 timestamp) internal {\n    // Sweep the old balance, if any\n    address[] memory users = new address[](1);\n    users[

In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train.csv',
    'val': 'vuln_data_val.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

import random
from datasets import concatenate_datasets, DatasetDict

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world""",
]

PROMPTS = [
    """The given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.
    
    This is the vulnerable functions:
    ```solidity
    {code}
    ```
    
This is the vulnerability explanation: 
{explanation}

Recommendation:
""",
]

def make_conversational(examples):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    recom = examples['vuln_recommendation']
    selected_system_prompt = random.choice(SYSTEM_PROMPT)
    selected_user_prompt = random.choice(PROMPTS) 
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code, explanation=desc)}]
    return { "conversations" : conversation, "label": recom}

dataset = dataset.map(make_conversational)

test_dataset = dataset['test']

In [ ]:
import re
from unsloth.chat_templates import get_chat_template
from tqdm import tqdm  # Import tqdm for progress bar
import pandas as pd

def extract_llm_res(raw_llm):
    # Regex pattern to extract the assistant's response after the "Vulnerability Explanation" section
    pattern = r'\[/INST](.*?)</s>'

    match = re.search(pattern, raw_llm, re.DOTALL)

    if match:
        assistant_response = match.group(1).strip()
        return assistant_response
    else:
        print(f"No response found. Response: {raw_llm}")

# Initialize the result collection
results = []

# Iterate over the dataset with tqdm for progress bar
for idx, example in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Processing dataset"):
    messages = example['conversations']  # Get the conversation for the current entry
    
    # Tokenize input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,  # Must add for generation
        return_tensors="pt",
    ).to("cuda")

    # Get LLM output
    outputs = model.generate(input_ids=inputs, max_new_tokens=2048, use_cache=True, repetition_penalty=1.2,
                                  temperature=0.1, min_p=0.1)
    raw_out = tokenizer.batch_decode(outputs)[0]
    
    # Extract LLM response
    llm_output = extract_llm_res(raw_out)
    
    # Get the ground truth for the current entry
    ground_truth = example['vuln_explanation']
    
    results.append({
        "vuln_code": example['vuln_code'],
        "ground_truth": ground_truth,
        "llm_output": llm_output
    })

# After the loop, you can convert the results list to a DataFrame or another suitable format if needed
verification_results_df = pd.DataFrame(results)


Processing dataset:   0%|                                                                          | 0/143 [00:00<?, ?it/s]


Processing dataset:   1%|▍                                                                 | 1/143 [00:02<04:44,  2.01s/it]


Processing dataset:   1%|▉                                                                 | 2/143 [00:05<06:09,  2.62s/it]


Processing dataset:   2%|█▍                                                                | 3/143 [00:07<05:26,  2.33s/it]


Processing dataset:   3%|█▊                                                                | 4/143 [00:12<08:24,  3.63s/it]

...


Processing dataset:  98%|██████████████████████████████████████████████████████████████▋ | 140/143 [07:59<00:07,  2.34s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████ | 141/143 [08:04<00:06,  3.17s/it]


Processing dataset:  99%|███████████████████████████████████████████████████████████████▌| 142/143 [08:06<00:02,  2.97s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [08:09<00:00,  2.88s/it]


Processing dataset: 100%|████████████████████████████████████████████████████████████████| 143/143 [08:09<00:00,  3.42s/it]

In [ ]:
verification_results_df.to_csv("sft_codellama13b_recommendation.csv")